# 🎨 ai-gen - LoRA Training & Image Generation

**Train LoRAs and generate ultra-realistic images with NO restrictions**

---

## 🚀 Quick Start

1. **Enable GPU**: Runtime → Change runtime type → T4 GPU
2. **Run Cell 1**: Setup (installs everything)
3. **Access GUI**: Click the ngrok URL when it appears

---

In [ ]:
# CELL 1: Complete Setup & Launch
# Run this cell - takes about 10-15 minutes

print("🚀 Starting MASUKA setup...\n")

# Clean up any previous installations to force fresh download
print("🧹 Cleaning previous installations...")
!rm -rf /content/CharForgex /content/ai-gen 2>/dev/null || true

# Clone the latest version from GitHub
print("📥 Downloading latest version from GitHub...")
!git clone https://github.com/SamuelD27/ai-gen.git /content/ai-gen

# Change to repo directory
import os
os.chdir('/content/ai-gen')

# Always pull latest changes to ensure we have the newest code
print("🔄 Ensuring latest version...")
!git pull origin main

# Download and run the setup script
print("⚙️  Running setup script...\n")
!python masuka_colab_setup.py

In [ ]:
# CELL 2: Clear Python Cache & Restart Backend
# Run this cell if you've updated code and need to reload changes

print("🧹 Clearing Python cache...")

# Remove all __pycache__ directories and .pyc files
!find /content/ai-gen -type d -name __pycache__ -exec rm -rf {} + 2>/dev/null || true
!find /content/ai-gen -type f -name "*.pyc" -delete 2>/dev/null || true

print("✓ Python cache cleared")

# Pull latest changes from git if connected
print("\n🔄 Pulling latest changes from repository...")
import os
os.chdir('/content/ai-gen')
!git pull origin main

print("\n✓ Ready to restart backend")
print("\n💡 To restart the backend:")
print("   1. Stop the current backend (if running): Press the stop button on Cell 1")
print("   2. Re-run Cell 1 to start with fresh code")
print("\nOR run this to restart automatically:")

# Kill existing backend process
!pkill -f "uvicorn app.main:app" 2>/dev/null || true

import time
time.sleep(2)

# Restart backend in background
print("\n🚀 Restarting backend...")
import subprocess
os.chdir('/content/ai-gen/charforge-gui/backend')
subprocess.Popen(['python', '-m', 'uvicorn', 'app.main:app', '--host', '0.0.0.0', '--port', '8000', '--reload'])

time.sleep(5)
print("✓ Backend restarted with fresh code!")
print("\n🌐 Frontend should reconnect automatically")
print("   If not, refresh your browser")